## Community Detection

### This notebook implements the following workflow for both the Erdos an CitNet datasets 

 1. Load in data as a networkX graph
 2. 

In [ ]:
# !pip install python-louvain
# !pip install networkx
# !pip install python-igraph

In [52]:
%%time
%pylab inline
import networkx as nx
import numpy as np
import matplotlib 
import scipy
import warnings

import community as community_louvain
import igraph

import operator

from PIL import Image
from numpy import asarray
from numpy import save
from scipy import misc
import glob

import matplotlib.pyplot as plt
from IPython.display import Image

from tqdm import tqdm
import time

import os, os.path

Populating the interactive namespace from numpy and matplotlib
CPU times: user 3.22 ms, sys: 15 µs, total: 3.24 ms
Wall time: 3.11 ms


#### Code for Unzipping songnet compressed. Run once.

In [44]:
# import gzip

# #Define the file's location
# file_path = './playlist-songnet-compressed.gz'

# #Open the file and read its contents
# with gzip.open(file_path, "rb") as file:
#     file_content = file.read()


# #Save the new txt file
# txt_file_name = "songnet.txt"

# with open(txt_file_name, "w") as file:
#     file.write(file_content)

# # import gzip
# # with gzip.open('./playlist-songnet-compressed.gz', 'rb') as f, open('songnet.txt', 'w') as f_out:
# #     f_out.write(f.read())

### Load in Validation spectrograms and reduced_songnet

In [45]:
val_path = '/datasets/home/21/321/ee228sp20ta1/G51/val_specs/'

In [46]:
file_name="./reduced_songnet.txt"

songs=nx.read_edgelist(file_name,create_using=nx.DiGraph())
node, edge=songs.order(),songs.size()
print("No. of nodes are=",node)
print("No. of edges are=",edge)

No. of nodes are= 4280
No. of edges are= 429918


### Helper function to find largest connected component

In [47]:
def find_largest_component(generator):
    sub_graphs = []
    for item in generator:
        sub_graphs.append(item)

    list_of_all_subgraphs = [(graph, len(graph.nodes)) for graph in sub_graphs]

    largest_count = 0
    for i in range(len(list_of_all_subgraphs)):
        count = list_of_all_subgraphs[i][1]
        if count > largest_count:
            largest_count = count
            largest_component = list_of_all_subgraphs[i][0]
    return largest_component

### Find Largest Connected Component

In [48]:
songs_ud = songs.to_undirected()
songs_ud_components = nx.connected_component_subgraphs(songs_ud)
songs_largest_component = find_largest_component(songs_ud_components)

### Find All Song ID's in the spectrogram validation set

In [49]:
songnet_ids = []
for image_path in tqdm(glob.glob(val_path + "/*.png")):
    
    
    # Load and collect song ids into list of strings
    ID = os.path.basename(image_path).partition(".png")[0]

    if ID in list(songs_largest_component.nodes):
        
        songnet_ids.append(ID)

100%|██████████| 2763/2763 [00:00<00:00, 7400.45it/s]


### Remove nodes from the network that aren't in the validation set

In [50]:
remove_nodes = []

for node in list(songs_largest_component.nodes):
    
    if node not in songnet_ids:
        remove_nodes.append(node)
        
songs_largest_component.remove_nodes_from(remove_nodes)

ids = list(songs_largest_component.nodes)

In [ ]:
# path, dirs, files = next(os.walk(val_path))
# file_count = len(files)
# print(file_count)

### Run Community_louvain

In [10]:
songs_community = community_louvain.best_partition(songs_largest_component)

In [11]:
songs_sorted = sorted_x = sorted(songs_community.items(), key=operator.itemgetter(1))
songs_coms = dict(songs_sorted)

songs_ud_degrees = songs_largest_component.degree()
songs_ud_degrees = dict(songs_ud_degrees)

## Save out largest connected component

In [14]:
nx.set_node_attributes(songs_largest_component, songs_coms, "community")
nx.set_node_attributes(songs_largest_component, songs_ud_degrees, "degrees")
# nx.set_node_attributes(songs_largest_component, song_partitions, "partitions")
nx.write_gml(songs_largest_component, "songnet_lc.gml")

#### Save out as JSON

In [12]:
#Save out community detection with louvain method
json = json.dumps(songs_coms)
f = open("val_communities_1.json","w")
f.write(json)
f.close()

### Run igraph community Detection

In [39]:
songs_lc = igraph.read('songnet_lc.gml', format="gml")
song_ptns = songs_lc.community_leading_eigenvector(clusters=16)
song_partitions = {}

for cluster in range(len(song_ptns)):
    for node in song_ptns[cluster]:
        idx = ids[node]
        song_partitions[idx] = cluster
        
#max(song_partitions.values())

### Save out igraph communities as JSON

In [42]:
# Save out community detection with igraph
json = json.dumps(song_partitions)
f = open("val_communities_10.json","w")
f.write(json)
f.close()



### TODO: Save out to format for Gephi Visualizer software

In [ ]:
# nx.set_node_attributes(songs_largest_component, songs_coms, "community")
# nx.set_node_attributes(songs_largest_component, songs_ud_degrees, "degrees")
# nx.set_node_attributes(songs_largest_component, song_partitions, "partitions")
# nx.write_gml(songs_largest_component, "songnet_lc.gml")

# Plot  Communities

In [ ]:
Image(filename = "Erdos_partitions.png")

### Compared to 'community' library, 'igraph' has more flexibilty to detect communities. Igraph allows the user to partition the network into the number of communities that the user wishes. Obviously this number is bounded. Now, you will use this feature to divide the given network into '5' communities using 'igraph' and observe the results. Write a python code to implement the above task for the Citation and the Erdos networks. Remember that unlike 'community', igraph provides multiple approaches to community detection, the most obvious approach being the greedy one because it optimizes modularity. Visualize your community detection results in Gephi for both the networks. Label the nodes in the visualization properly. Use largest connected components if required. Use different colors for nodes in every community. Include the images(.jpg, .png) of your visualizations here.

# TODO? Analyze communities

In [ ]:
# track={}

# for key,value in erdos_community.items():
#     if value not in track:
#         track[value]=0
#     else:
#         track[value]+=1
        
# import operator

# sorted_track = sorted(track.items(), key=operator.itemgetter(1))

# erdos_community_counts = dict(sorted_track)

# top_five = []
# for i in range(5):
#     largest = erdos_community_counts.popitem()    
#     top_five.append(largest[0])
    

# top_five_com = {}
# for community in top_five:
#     top_five_com[community] = [x for x,y in erdos_largest_component.nodes(data=True) if y['community']==community]
    
# from collections import defaultdict
# top_three_nodes_per_community = {}
# com_with_degree = defaultdict(list)

# for cm, list_nodes in top_five_com.items():
#     test = []
#     for node in list_nodes:
#         degree = (erdos_largest_component.nodes[node]['author'], erdos_largest_component.nodes[node]['degrees'])
#         test.append(degree)
#     sorted_list = dict(sorted(test, key = lambda x: x[1]))
    
#     top_3 = []
#     for i in range(3):
#         largest = sorted_list.popitem()    
#         top_3.append(largest)
    
#     com_with_degree[cm] = top_3
    
    